In [1]:
import pandas as pd
import numpy as np

path = '../data/ATAUSDT_5m.csv'
symbol = path.split('/')[-1].split('_')[0].replace('USDT', '')
df = pd.read_csv(path)
df.head()

,time,open,high,low,close,volume
0,2023-08-01 00:00:00,0.0806,0.0806,0.0805,0.0805,62108.0
1,2023-08-01 00:05:00,0.0805,0.0805,0.0801,0.0802,412818.0
2,2023-08-01 00:10:00,0.0803,0.0804,0.0803,0.0804,169956.0
3,2023-08-01 00:15:00,0.0804,0.0805,0.0804,0.0805,260639.0
4,2023-08-01 00:20:00,0.0805,0.0807,0.0804,0.0805,329873.0


In [2]:
import pandas_ta as ta

temp = df.ta.macd()
df['MACD'] = temp['MACD_12_26_9']   
df['MACD_SIGNAL'] = temp['MACDs_12_26_9']
df['MACD_HIST'] = temp['MACDh_12_26_9']
temp = df.ta.cdl_pattern(name="longline")
df['LONG_LINE'] = temp['CDL_LONGLINE']
#buy LONG_LINE == 100 and MACD_HIST > 0
df['Signal_Buy'] = np.where((df['LONG_LINE'] == 100) & (df['MACD_HIST'] > 0), 2, 0)
#sell LONG_LINE == -100 and MACD_HIST < 0
df['Signal_Sell'] = np.where((df['LONG_LINE'] == -100) & (df['MACD_HIST'] < 0), 1, 0)
df['Signal'] = df['Signal_Buy'] + df['Signal_Sell']
df['ATR'] = df.ta.atr()
df['ADX'] = df.ta.adx()['ADX_14']
#help(ta.adx)

In [3]:
df.ta.cdl_pattern(name="longline").value_counts()

CDL_LONGLINE
 0.0            19069
 100.0           2288
-100.0           2143
dtype: int64

In [4]:
help(ta.mad)

Help on function mad in module pandas_ta.statistics.mad:

mad(close, length=None, offset=None, **kwargs)
    Rolling Mean Absolute Deviation
    
    Sources:
    
    Calculation:
        Default Inputs:
            length=30
        mad = close.rolling(length).mad()
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 30
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type of fill method
    
    Returns:
        pd.Series: New feature generated.



In [5]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,MACD,MACD_SIGNAL,MACD_HIST,LONG_LINE,Signal_Buy,Signal_Sell,Signal,ATR,ADX
33,2023-08-01 02:45:00,0.0789,0.0790,0.0788,0.0789,414641.0,-0.000490,-0.000453,-3.735350e-05,0.0,0,0,0,0.000335,63.371775
34,2023-08-01 02:50:00,0.0789,0.0789,0.0788,0.0788,198657.0,-0.000485,-0.000459,-2.514476e-05,0.0,0,0,0,0.000318,63.384123
35,2023-08-01 02:55:00,0.0788,0.0790,0.0788,0.0790,348317.0,-0.000458,-0.000459,7.772857e-07,0.0,0,0,0,0.000309,62.704106
36,2023-08-01 03:00:00,0.0789,0.0791,0.0787,0.0790,599179.0,-0.000433,-0.000454,2.117037e-05,0.0,0,0,0,0.000316,61.432379
37,2023-08-01 03:05:00,0.0790,0.0791,0.0789,0.0791,510844.0,-0.000400,-0.000443,4.336250e-05,0.0,0,0,0,0.000308,60.289824


In [6]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

C:\Users\hiban\AppData\Local\Temp\ipykernel_24460\1852176384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


,open,high,low,close,volume,MACD,MACD_SIGNAL,MACD_HIST,LONG_LINE,Signal_Buy,Signal_Sell,Signal,ATR,ADX
time,,,,,,,,,,,,,,
2023-08-01 02:45:00,0.0789,0.0790,0.0788,0.0789,414641.0,-0.000490,-0.000453,-3.735350e-05,0.0,0,0,0,0.000335,63.371775
2023-08-01 02:50:00,0.0789,0.0789,0.0788,0.0788,198657.0,-0.000485,-0.000459,-2.514476e-05,0.0,0,0,0,0.000318,63.384123
2023-08-01 02:55:00,0.0788,0.0790,0.0788,0.0790,348317.0,-0.000458,-0.000459,7.772857e-07,0.0,0,0,0,0.000309,62.704106
2023-08-01 03:00:00,0.0789,0.0791,0.0787,0.0790,599179.0,-0.000433,-0.000454,2.117037e-05,0.0,0,0,0,0.000316,61.432379
2023-08-01 03:05:00,0.0790,0.0791,0.0789,0.0791,510844.0,-0.000400,-0.000443,4.336250e-05,0.0,0,0,0,0.000308,60.289824


In [7]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [8]:
df['Signal'].value_counts()

0    20869
2     1347
1     1251
Name: Signal, dtype: int64

In [9]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class Scalping_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5
    save_size_rate = 0.125
    init_cash = 350
    profit_target = 500
    size_multiplier_rate = 0.5

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'Signal')

    def next(self):
        super().next()

        slatr = self.atr_sl_rate * self.data.ATR[-1]
        size_multiplier = self.size_multiplier_rate if self.equity >= self.profit_target else 1.0

        if self.signal == 2:
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity > self.init_cash * 1.5:
                self.buy(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.buy(sl=sl_price, tp=tp_price)

        if self.signal == 1:
            sl_price = self.data.Close[-1] + slatr
            tp_price = self.data.Close[-1] - self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity >= self.init_cash * 1.5:
                self.sell(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.sell(sl=sl_price, tp=tp_price)
            

bt = Backtest(df, Scalping_Strategy, cash=350, commission=.00002, margin=.05)
stat = bt.run()
stat

Start                     2023-08-01 02:45:00
End                       2023-10-21 14:15:00
Duration                     81 days 11:30:00
Exposure Time [%]                   48.881408
Equity Final [$]                    99.974498
Equity Peak [$]                   2545.687565
Return [%]                         -71.435858
Buy & Hold Return [%]                -4.18251
Return (Ann.) [%]                  -99.928525
Volatility (Ann.) [%]     4980744018504.72...
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -98.586026
Avg. Drawdown [%]                  -13.500683
Max. Drawdown Duration       28 days 13:30:00
Avg. Drawdown Duration        1 days 10:59:00
# Trades                                 1838
Win Rate [%]                        36.670294
Best Trade [%]                       2.698349
Worst Trade [%]                     -1.579017
Avg. Trade [%]                    

In [10]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stat['_equity_curve'].index, y=stat['_equity_curve']['Equity'])])
fig.show()

In [11]:
%%time
stats = bt.optimize(atr_sl_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    TPSL_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    size_multiplier_rate = np.arange(0.1, 0.5, 0.1).tolist(),
                    maximize='Equity Final [$]')
stats

C:\Users\hiban\AppData\Roaming\Python\Python310\site-packages\backtesting\backtesting.py:1488: UserWarning:

Searching for best of 576 configurations.



  0%|          | 0/12 [00:00<?, ?it/s]

CPU times: total: 4min 51s
Wall time: 7min 5s


Start                     2023-08-01 02:45:00
End                       2023-10-21 14:15:00
Duration                     81 days 11:30:00
Exposure Time [%]                   36.050624
Equity Final [$]                 10612.107135
Equity Peak [$]                  11345.908621
Return [%]                         2932.03061
Buy & Hold Return [%]                -4.18251
Return (Ann.) [%]             52668314.111693
Volatility (Ann.) [%]     168440448978.459167
Sharpe Ratio                         0.000313
Sortino Ratio                   262390.476164
Calmar Ratio                    669235.075191
Max. Drawdown [%]                  -78.699273
Avg. Drawdown [%]                   -6.575939
Max. Drawdown Duration       29 days 10:30:00
Avg. Drawdown Duration        0 days 17:41:00
# Trades                                 2398
Win Rate [%]                        52.627189
Best Trade [%]                       1.258163
Worst Trade [%]                     -1.579017
Avg. Trade [%]                    

In [12]:
stats.keys()

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [13]:
stats._strategy

<Strategy Scalping_Strategy(atr_sl_rate=1.3,TPSL_rate=0.7,size_multiplier_rate=0.4)>

In [14]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stats['_equity_curve'].index, y=stats['_equity_curve']['Equity'])])
fig.update_layout(title='Profit |'+symbol, yaxis_title='USDT', xaxis_title='Date')
fig.show()